<a href="https://colab.research.google.com/github/Manwa94/Boston_House_Price_Prediction/blob/main/Text_classification_Using_Bert_%26_tenserflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

import warnings
warnings.filterwarnings('ignore') 

In [7]:
import pandas as pd
df=pd.read_csv("spam.csv")
df.head(5)

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [8]:
df.groupby('Category').describe()

Message                                                            \
           count unique                                                top   
Category                                                                     
ham         4827   4518                             Sorry, I'll call later   
spam         747    653  Please call our customer service representativ...   

               
         freq  
Category       
ham        30  
spam        4

In [9]:
df['spam']=df['Category'].apply(lambda x:1 if x=='spam' else 0)
df.head(4)

,Category,Message,spam
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0


In [37]:
y_test.value_counts()

0    962
1    153
Name: spam, dtype: int64

In [36]:
y_train.value_counts()

0    3865
1     594
Name: spam, dtype: int64

In [44]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(df['Message'],df['spam'])


In [43]:
149/966

0.15424430641821946

In [34]:
len(X_train)

4459

In [35]:
len(X_test)

1115

In [16]:
bert_preprocess=hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')
bert_encoder=hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4')

In [19]:
def get_sentence_embeding(sentences):
  preprocessed_text=bert_preprocess(sentences)
  return bert_encoder(preprocessed_text)['pooled_output']

In [20]:
get_sentence_embeding([
    "500$ discount. hurry up",
    "Bhavin, are you up for a volleybal game tomorrow?"
])

<tf.Tensor: shape=(2, 768), dtype=float32, numpy=
array([[-0.8435169 , -0.51327264, -0.88845724, ..., -0.74748874,
        -0.75314736,  0.91964483],
       [-0.8720836 , -0.5054398 , -0.9444668 , ..., -0.85847515,
        -0.7174535 ,  0.88082975]], dtype=float32)>

In [21]:
e=get_sentence_embeding([
    "banana",
    "grapes",
    "mango",
    "jeff bezos",
    "elon musk"
])

In [25]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_similarity([e[3]],[e[4]])

array([[0.9872034]], dtype=float32)

In [27]:
text_input=tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')

preprocessed_text=bert_preprocess(text_input)
outputs=bert_encoder(preprocessed_text)

l=tf.keras.layers.Dropout(0.3, name="dropout")(outputs['pooled_output'])
l=tf.keras.layers.Dense(1, activation="sigmoid", name="output")(l)

model=tf.keras.Model(inputs=[text_input],outputs=[l])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer_2 (KerasLayer)     {'input_word_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_type_ids':                                                
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128)}                                                      

In [28]:
model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])

In [29]:
model.fit(X_train,y_train,epochs=5)

Epoch 1/5
131/131 [==============================] - 2064s 16s/step - loss: 0.3391 - accuracy: 0.8660
Epoch 2/5
131/131 [==============================] - 2092s 16s/step - loss: 0.2414 - accuracy: 0.8952
Epoch 3/5
131/131 [==============================] - 2043s 16s/step - loss: 0.2065 - accuracy: 0.9167
Epoch 4/5
131/131 [==============================] - 2026s 15s/step - loss: 0.1845 - accuracy: 0.9289
Epoch 5/5
131/131 [==============================] - 2029s 15s/step - loss: 0.1682 - accuracy: 0.9359


In [30]:
model.evaluate(X_test,y_test)

44/44 [==============================] - 688s 16s/step - loss: 0.1558 - accuracy: 0.9390


[0.1557673215866089, 0.9390243887901306]

In [32]:
reviews=['Reply to win A$100 weekly! where will the 2006 FIFA world cup be held? Sent STOP to 87239 to end',
'You are awarded a SiPix Digital Camera! call 9921278527 from landline. Delivery within 28days.',
'It to 88488. Your 500 free text messages are valid until 31 December 2005',
'Hey Sam, Are you coming for a cricket game tomorrow',
"Why don't you wait til at least wednesday to see if you get your."
]
model.predict(reviews)

1/1 [==============================] - 4s 4s/step


array([[0.5262969 ],
       [0.5646099 ],
       [0.55800784],
       [0.02671521],
       [0.01591785]], dtype=float32)